In [1]:
import face_recognition
import os
from pathlib import Path
from shutil import copyfile
import tkinter as tk
from tkinter import filedialog
import cv2
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import zipfile  

In [2]:
cwd = os.getcwd()
p = os.path.join(cwd, 'dataset' )

In [3]:
results_path = os.path.join(cwd, 'results')
encodings = {}
curr_image_cluster_id = None

In [4]:
# SMTP email configuration
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587
SMTP_USERNAME = 'rashisolanki2002@gmail.com'
SMTP_PASSWORD = 'ttes rjqa dlei phsh'

In [5]:
def process_file(filepath):
    img = face_recognition.load_image_file(filepath)
    fe = face_recognition.face_encodings(img)
    if fe:
        fe = fe[0]
    else:
        return
    action_taken = False
    curr_image_cluster_id = None
    for cluster_id, cluster_encodings in encodings.items():
        results = face_recognition.compare_faces(cluster_encodings, fe)
        if all(results):
            curr_image_cluster_id = cluster_id
            encodings.get(cluster_id).append(fe)
            action_taken = True

    if not action_taken:
        curr_image_cluster_id = "cluster_%s" % (len(encodings.keys()) + 1)
        print("creating new cluster %s" % curr_image_cluster_id)
        encodings[curr_image_cluster_id] = [fe]
    curr_cluster = os.path.join(results_path, curr_image_cluster_id)
    curr_cluster_dir = Path(curr_cluster)
    curr_cluster_dir.mkdir(parents=True, exist_ok=True)
    copyfile(filepath, os.path.join(curr_cluster_dir, os.path.basename(filepath)))

In [7]:
def send_email(subject, body, to_email, attachment_path=None):
    
    msg = MIMEMultipart()
    msg['From'] = SMTP_USERNAME
    msg['To'] = to_email
    msg['Subject'] = subject
    msg.attach(MIMEText(body, 'plain'))

    if attachment_path:
        if os.path.isfile(attachment_path):
            attachment = open(attachment_path, 'rb')
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(attachment.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f'attachment; filename= {os.path.basename(attachment_path)}')
            msg.attach(part)
        else:
            print("Attachment file not found:", attachment_path)

    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(SMTP_USERNAME, SMTP_PASSWORD)
        text = msg.as_string()
        server.sendmail(SMTP_USERNAME, to_email, text)
        server.quit()
        print("Email sent successfully.")
    except Exception as e:
        print("Error sending email:", str(e))

In [8]:
# Function to process a single image and assign it to a cluster
def process_image(img):
    global curr_image_cluster_id

    # Detect faces in the image
    face_locations = face_recognition.face_locations(img)

    if not face_locations:
        return

    fe = face_recognition.face_encodings(img, known_face_locations=face_locations)[0]

    action_taken = False

    for cluster_id, cluster_encodings in encodings.items():
        results = face_recognition.compare_faces(cluster_encodings, fe)
        print("results %s %s" % (results, cluster_id))
        if all(results):
            print("cluster_id %s" % cluster_id)
            curr_image_cluster_id = cluster_id
            encodings.get(cluster_id).append(fe)
            action_taken = True

    if not action_taken:
        curr_image_cluster_id = "cluster_%s" % (len(encodings.keys()) + 1)
        print("creating new cluster %s" % curr_image_cluster_id)
        encodings[curr_image_cluster_id] = [fe]

    return curr_image_cluster_id

In [ ]:
curr = 0
for subdir, dirs, files in os.walk(p):
    total = len(files)
    for file in files:
        filepath = os.path.join(subdir, file)
        print("File: %s" % filepath)
        process_file(filepath)
        curr += 1
        print("file %s/%s - %s encodings" % (curr, total, len(encodings)))
        print()

File: /Users/rashisolanki/Downloads/face-clustering/dataset/d 4.jpg
file 1/33 - 0 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/d 2.jpg
file 2/33 - 0 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/d 3.jpg
file 3/33 - 0 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/d 1.jpg
file 4/33 - 0 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/sir 1.jpg
file 5/33 - 0 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/sir 2.jpg
file 6/33 - 0 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/o 8.jpg
file 7/33 - 0 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/o 5.jpg
file 8/33 - 0 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/p 1.jpg
file 9/33 - 0 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/k 1.jpg
file 10/33 - 0 encodings

File: /Users/rashisolanki/Downloads/face-clustering/dataset/r 3.j

In [7]:
print("There are %s different people" % len(encodings.keys()))

There are 8 different people


In [10]:
# Initialize OpenCV VideoCapture for webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    # Convert the frame to RGB format (required by face_recognition)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the detected face and get the cluster ID
    cluster_id = process_image(rgb_frame)

    # Display cluster ID on the frame
    if cluster_id:
        cv2.putText(frame, cluster_id, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Webcam', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

results [True, True, True, True, True, True, True, True, True, True, True, True] cluster_1
cluster_id cluster_1
results [True, True, True, True, True, True, True, True, True, True, True, True, True] cluster_1
cluster_id cluster_1
results [True, True, True, True, True, True, True, True, True, True, True, True, True, True] cluster_1
cluster_id cluster_1
results [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True] cluster_1
cluster_id cluster_1
results [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True] cluster_1
cluster_id cluster_1
results [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True] cluster_1
cluster_id cluster_1
results [True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True] cluster_1
cluster_id cluster_1
results [True, True, True, True, True, True, True, True, True, True, True, True, True, Tru

In [11]:
# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()

In [12]:
recipient_email = input("Enter the recipient's email address: ")

Enter the recipient's email address: rashimahavir.solanki2020@vitstudent.ac.in


In [14]:
# Create a zip archive of the cluster directory
email_subject = "Pictures from the event"
email_body = "Thank you for attending the event. Here is a folder of your pictures."
cluster_directory = os.path.join(results_path, cluster_id)
zip_filename = f"{cluster_id}.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(cluster_directory):
        for file in files:
            file_path = os.path.join(root, file)
            arcname = os.path.relpath(file_path, cluster_directory)
            zipf.write(file_path, arcname=arcname)
send_email(email_subject, email_body, recipient_email, zip_filename)

Email sent successfully.


SyntaxError: 'break' outside loop (3419260034.py, line 13)